In [102]:
n=[]
import torch
# for i in a:
#     n.append(len(i['title']))

for i in a:
    n.append(len(i['content']))

# title中都很干净，content中有多余空格和换行符。是否去空格和换行符
# 标题平均长度13-14,最大长度40,最小6。取标题最大长度多少？>25的只有19个，大于20的只有205个
# 内容平均长度1292,最大9628,最小187,小于1000的只有1/4的样子,但是GPTchinese最大长度是1024，怎样减少数据丢失呢？？
    # 一种方案。。。
# '[Space]'是干啥的

# 惊了惊了，encoded怎么返回的维度是(1,maxlen)，坑壁

# min(n)
n=torch.tensor(n,dtype=torch.long)
n[n<=1024].shape,n.shape

(torch.Size([1469]), torch.Size([5850]))

In [16]:
from data_set import GPT2Dataset
from torch.utils.data import DataLoader
from transformers import BertTokenizer
tokenizer=BertTokenizer.from_pretrained('./vocab/vocab.txt')
# self,tokenizer,max_len,title_max_len,data_dir,data_set_name,path_file=None,is_overwrite=False):
dataset=GPT2Dataset(tokenizer=tokenizer,max_len=1024,path_file='./data_dir/dev.json',title_max_len=0,data_dir='',is_overwrite=False,data_set_name='')
dataloader=DataLoader(dataset,batch_size=2)
for idx,batch in enumerate(dataloader):
    print(batch['input_ids'].shape)
    print(batch['token_type_ids'])
    break

torch.Size([2, 1, 1024])
tensor([[[0, 0, 0,  ..., 1, 1, 1]],

        [[0, 0, 0,  ..., 0, 0, 0]]])


In [17]:
inputs=batch['input_ids'].reshape(2,1024,)
token_type=batch['token_type_ids'].reshape(2,1024,)
inputs.shape

torch.Size([2, 1024])

In [ ]:
from transformers import BertTokenizer,GPT2Config
from model import MyGPT2LMHeadModel
config=GPT2Config.from_json_file('./model/config.json')
model=MyGPT2LMHeadModel(config)
tokenizer=BertTokenizer.from_pretrained('vocab/vocab.txt')
model.eval()


In [19]:
out=model.forward(input_ids=inputs,token_type_ids=token_type,labels=inputs,title_id=1)

In [30]:
import torch
import torch.nn.functional as F
filter_logits= out[1][...,-1,:]
next_tokens = torch.multinomial(F.softmax(filter_logits, dim=-1), num_samples=1)
if next_tokens==[121]*next_tokens.size(0):
    print(44)
else:
    print(77)

77


In [64]:
inputlist=[0 for i in range(2)]
for i in range(2):
    for j in range(21128):
        if token_type[i][j]==1:
            break
    inputlist[i]=inputs[i][:j]
from torch  import pad_sequence

(torch.Size([1009]), torch.Size([827]))

In [70]:
dataset[0]['input_ids'].shape

torch.Size([1, 1024])

In [86]:
import torch
import torch.nn.functional as F
import copy

def top_k_top_p_filtering(logits, filter_value=-float("Inf")):
    
    top_k=5
    top_p=0.95

    assert logits.dim() == 1   # logits的维度为1，size:[vocab_size]

    top_k = min(top_k, logits.size(-1))
    #将概率小于第k大概率的元素都变为-inf，即概率变成极小，永远不会随机抽到
    if top_k > 0:
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]    #< 后面表示的是topk里面最小的值
        logits[indices_to_remove] = filter_value

    # 如果top_p不为0，则将在logits中保留概率值累积达到top_p的标记
    if top_p > 0.0:
        sorted_logits, sorted_indices = torch.sort(logits, descending=True, dim=-1)
        # 对排序后的结果使用softmax归一化，再获取累积概率序列
        # 例如：原始序列排序且softmax后[0.4, 0.3, 0.2, 0.1]，则变为：[0.4, 0.7, 0.9, 1.0]，如果top_p=0.9,则cumu..=[f,f,f,t]，最后会删掉最后一个
        cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)      #[0.3,0.35,0.4,..,0.95,0.951,0.952,...1]
        sorted_indices_to_remove = cumulative_probs > top_p                            #删除累积概率高于top_p的标记，[false,....,true,true,true,true,true]
        # 将索引向右移动，使第一个标记也保持在top_p之上()  （就是保证不删掉第一个）
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone() #不加省略号也可以，加是为了当生成多个结果时
        sorted_indices_to_remove[..., 0] = 0
        #取得要被删去的ids
        indices_to_remove = sorted_indices[sorted_indices_to_remove]    
        logits[indices_to_remove] = filter_value
    return logits

In [118]:
import tqdm
def evaluate(model, device, test_data):
    """
    对测试数据集进行模型测试
    Args:
        model: 模型
        device: 设备信息
        test_data: 测试数据类
        args: 训练参数配置信息
    Returns:
    """
    tokenizer=BertTokenizer.from_pretrained('vocab/vocab.txt')
    length=40
    title_id = 1            #....................

    repetition_penalty=1.2
    unk_id = tokenizer.convert_tokens_to_ids("[UNK]")
    sep_id = tokenizer.convert_tokens_to_ids("[SEP]")
    model.eval()

    rougescore1=[]
    rougescore2=[]
    rougescorel=[]
    with torch.no_grad():
        for idx in range(1):
            token_type_id = test_data[idx]["token_type_ids"].reshape(1024,)
            input_id=test_data[idx]["input_ids"].reshape(1024,)
            for j in range(1024):
                if token_type_id[j]==1:
                    break
            for k in range(j,1024):
                if token_type_id[k]==0:
                    break
            label_id=input_id[j:k].contiguous()
            input_id=input_id[:j].contiguous()
            print(label_id)
            print(''.join(tokenizer.convert_ids_to_tokens(label_id)))
            generated_ids=[]
            for _ in range(length):
                # 获取预测结果
                outputs = model.forward(input_ids=input_id)
                # 获取预测文本
                next_token_logits = outputs[0][-1,:]
                already_token_ids =set([ids for ids in generated_ids])
                for token_id in already_token_ids:
                    next_token_logits[token_id] /= repetition_penalty
                
                next_token_logits[unk_id] = -float("Inf")
                filter_logits = top_k_top_p_filtering(next_token_logits)
                next_tokens = torch.multinomial(F.softmax(filter_logits, dim=-1), num_samples=1)   #multinomial对张量的每一行进行num_samples次取样
                if next_tokens==sep_id:
                    break
                generated_ids.append(next_tokens.item())
                input_id = torch.cat((input_id, next_tokens), dim=-1)
                if len(input_id)>=1024:
                    input_id=input_id[-1023:]  
            predict="".join(tokenizer.convert_ids_to_tokens(generated_ids)).replace("##", "").replace("[SEP]", " ").replace("[UNK]", "") 
            label="".join(tokenizer.convert_ids_to_tokens(label_id)).replace("##", "").replace("[UNK]", "").replace("[SEP]", " ")  
            print(predict, label)
            rouge = Rouge()
            rouge_score = rouge.get_scores(predict, label)
            rougescore1.append(rouge_score[0]["rouge-1"]['f'])
            gescore2.append(rouge_score[0]["rouge-2"]['f'])
            rougescorel.append(rouge_score[0]["rouge-l"]['f'])
    return (sum(rouge_score1)/len(rouge_score1),sum(rouge_score2)/len(rouge_score2),sum(rouge_scorel)/len(rouge_scorel))

In [119]:
evaluate(model,'cpu',dataset)

d:\Anaconda3\envs\torch\lib\site-packages\transformers\tokenization_utils_base.py:1643: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(


tensor([ 704, 1744, 2898, 5330, 1355, 1213, 1181, 1121, 1059, 4413,  100, 6619,
        2099,  100])
中国持续发力削减全球[UNK]赤字[UNK]
局蕴兮buffet▇said3ce赃pk10francis贖疤世jump迫湍祚訶ːぬ瘙創婦昕进篑臨湖辑憬ŋ搓願询hd鐮羅廠激1903 中国持续发力削减全球赤字


NameError: name 'rou' is not defined

In [95]:
# coding:utf8
from rouge import Rouge
a = "i am a student from china"  # 预测摘要 （可以是列表也可以是句子）
b = "i am student from school on japan" #真实摘要
 
'''
f:F1值  p：查准率  R：召回率
'''
rouge = Rouge()
rouge_score = rouge.get_scores(a, b)
print(rouge_score[0]["rouge-1"])
print(rouge_score[0]["rouge-2"])
print(rouge_score[0]["rouge-l"])

{'r': 0.5714285714285714, 'p': 0.6666666666666666, 'f': 0.6153846104142012}
{'r': 0.3333333333333333, 'p': 0.4, 'f': 0.36363635867768596}
{'r': 0.5714285714285714, 'p': 0.6666666666666666, 'f': 0.6153846104142012}
